In [31]:
## necessary packages ##
#using Pkg
#Pkg.add("Distances")

#using Distributions
using Random
using Distances
using Distributions

In [32]:
# Set the parameters of the simulated data #

N = 2000;   # sample size
p = 2;      # No. covariates
β = [1.0, -2.0]; #regression coeff
σ2 = 1.0; ϕ = 9.0; ν = 0.5; τ2 = 1.0; # hyperparmeters in matern

In [20]:
# Generate simulation data #

Random.seed!(123);
N = 1200;  #
n = 1000;  # No. of observations
coords = rand(N, 2); # random location over unit square
X = hcat(fill(1, (N,)), rand(Normal(), N));                   # design matrix
D = pairwise(Euclidean(), coords, dims = 1);                  # distance matrix
Cov = σ2 * exp.(-ϕ * D);                                      # covariance matrix
z = rand(MvNormal(Cov), 1);                                   # latent process
y = X * β + z + sqrt(τ2) * rand(Normal(), N);                 # response process

In [34]:
sqrt(3)

1.7320508075688772

In [ ]:
# Split the data for training and testing #
Random.seed!(123);
ordx = 1:N; #sortperm(coords[:, 1]);                          # 
X_ord = X[ordx, :]; Y_ord = Y[ordx, :]; ω_ord = ω[ordx, :];   # sorted data
ω_incp_obs = ω_ord + fill(1.0, (N, 1)) * transpose(β[1, :]);  # latent process + intercept
coords_ord = coords[ordx, :];
S = sample(1:N, n, replace = false, ordered = true);          # observed location index for 1st response
M = setdiff(1:N, S);                                          # index of missing data


# build neighbors #

m = 10;                                                       # number of nearest neighbor                       
NN = BuildNN(coords_ord[S, :], m);                            # build nearest neighbor 
nnIndx_col = vcat(NN.nnIndx, 1:n);                            # the index of columns
nnIndx_row = zeros(Int64, 0);                                               
for i in 2:m
    nnIndx_row = vcat(nnIndx_row, fill(i, i-1))
end
nnIndx_row = vcat(nnIndx_row, repeat((m + 1):n, inner = m), 1:n);  # the index of rows

# check neighbors
index = 660
gr()
scatter(coords_ord[S, 1], coords_ord[S, 2], xlim = (0, 1), ylim = (0, 1), color = :white)
scatter!(coords_ord[S[1:index], 1], coords_ord[S[1:index], 2], color = :grey)
scatter!(coords_ord[S[index:index], 1], coords_ord[S[index:index], 2], color = :red)
scatter!(coords_ord[S[NN.nnIndx[NN.nnIndxLU[index]:(NN.nnIndxLU[index + 1] - 1)]], 1], 
    coords_ord[S[NN.nnIndx[NN.nnIndxLU[index]:(NN.nnIndxLU[index + 1] - 1)]], 2], color = :orange)


@save "sim1data.jld" 